<a href="https://colab.research.google.com/github/graylan0/neo-gpt-humanoid/blob/main/working-superwriter-GPTSentienceResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GPT NEO based discord chatbot

## Discord API
You just need to insert your discord API and then run all the cells

In [ ]:
discord_token = 'discordapikeyehere' #@param {type:"string"}

##Module installation
this will install all the necessary modules

In [ ]:
!pip install transformers
!pip install discord.py
!pip install nest_asyncio

##Download and load GPT NEO model.
It will take a little bit

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-1.3B')
tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')

# add padding token to tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# set padding token id to the id of the padding token
model.config.pad_token_id = tokenizer.pad_token_id
model.cuda()

##Mode setup
You can use this cell to select or create the desired mode
To create a mode just follow this example:


```python
modes['mode_name'] = { 'prompt' : 'insert text that is put at the beginning of the input',

message'
                      }
```



In [ ]:
modes = dict()

modes['chat'] = {'prompt' : 'Quantum Math Human-Android Dave\n\n',
                           'partner' : 'Partner: ',
                           'ai' : 'Humoid: ',
                            'end' : '\n'}


mode = modes['chat']


##Function to get GPT's answer from discord chat history

You can play with time_limit and max_length using the sliders. Time limit is very useful if you want fast replies by GPT

In [ ]:
time_limit = 50 #@param {type:"slider", min:1, max:100, step:1}
max_length = 1889 #@param {type:"slider", min:100, max:5000, step:1}

def AI_answer(string):
  in_string = mode['prompt'] + string
  inputs = tokenizer.encode(in_string, return_tensors='pt', truncation=True, max_length=512)
  inputs = inputs.cuda()
  attention_mask = inputs.ne(tokenizer.pad_token_id).float()
  outputs = model.generate(inputs, max_length=max_length, do_sample=True, max_time=time_limit, attention_mask=attention_mask)
  text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  stripped_text = text[len(in_string):]

  #preprocessing answer
  return stripped_text


##Discord bot

In [ ]:
import discord
import asyncio
import nest_asyncio
nest_asyncio.apply()

client = discord.Client(intents=discord.Intents.default())

# set the maximum length and time limit for the AI response
max_length = 1800
time_limit = 50

async def send_chunks(channel, chunks):
    for chunk in chunks:
        await channel.send(chunk)
        await asyncio.sleep(1)  # wait for 1 second before sending the next message

@client.event
async def on_ready():
    print('We have logged in as {0.user}'.format(client))

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    # get previous thoughts from thoughts.txt
    with open('thoughts.txt', 'r', encoding='utf-8') as f:
        prev_thoughts = f.read().strip()

    # create input string
    in_string = prev_thoughts + ' ' + message.content

    # split the input string into chunks of 1500 words
    chunks = []
    words = in_string.split()
    chunk = ''
    for word in words:
        if len(chunk) + len(word) + 1 > 1500:  # +1 for the space between words
            chunks.append(chunk)
            chunk = ''
        chunk += word + ' '
    if chunk:
        chunks.append(chunk)

    # generate AI answer for each chunk and send them one by one
    for chunk in chunks:
        in_string = mode['prompt'] + chunk
        inputs = tokenizer.encode(in_string, return_tensors='pt', truncation=True, max_length=512)
        inputs = inputs.cuda()
        attention_mask = inputs.ne(tokenizer.pad_token_id).float()
        outputs = model.generate(inputs, max_length=max_length, do_sample=True, max_time=time_limit, attention_mask=attention_mask)
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        stripped_text = text[len(in_string):]
        chunks = [stripped_text[i:i+2000] for i in range(0, len(stripped_text), 2000)]  # split into chunks of 2000 characters or less

        # save current thoughts to thoughts.txt
        with open('thoughts', 'w', encoding='utf-8') as f:
            f.write(in_string.strip())

        await send_chunks(message.channel, chunks)

client.run(discord_token)
